In [1]:
import GridMLM_tokenizers
from GridMLM_tokenizers import CSGridMLMTokenizer
from models_graph import make_graph_from_input_ids, remove_consecutive_duplicates, remove_out_of_dict_ids, compute_edge_features
from data_utils import CSGridMLMDataset
import torch
from tqdm import tqdm

/home/maximos/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = CSGridMLMTokenizer(fixed_length=256)
chord_features = GridMLM_tokenizers.CHORD_FEATURES
chord_id_features = {tokenizer.vocab[k]: v for k, v in chord_features.items()}
print(chord_features)

{'C:maj': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], 'C#:maj': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], 'D:maj': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0], 'D#:maj': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], 'E:maj': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1], 'F:maj': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], 'F#:maj': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], 'G:maj': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1], 'G#:maj': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0], 'A:maj': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], 'A#:maj': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], 'B:maj': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1]

In [3]:
print(chord_features['C:maj'])
print(chord_features['D:maj'])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]


In [4]:
print(chord_id_features[7])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]


In [5]:
train_dir = '/mnt/ssd2/maximos/data/hooktheory_midi_hr/CA_train'
val_dir = '/mnt/ssd2/maximos/data/hooktheory_midi_hr/CA_test'
# train_dir = '/media/maindisk/data/hooktheory_midi_hr/CA_train'
# val_dir = '/media/maindisk/data/hooktheory_midi_hr/CA_test'

In [6]:
# train_dataset = GuidedGridMLMDataset(train_dir, tokenizer, 512, frontloading=True)
val_dataset = CSGridMLMDataset(val_dir, tokenizer, frontloading=True, name_suffix='Q4_L80_bar_PC')

Loading data file.


In [7]:
print(val_dataset[0])

{'harmony_ids': [6, 269, 269, 269, 269, 6, 152, 152, 66, 66, 6, 269, 269, 269, 269, 6, 152, 152, 66, 66, 6, 269, 269, 269, 269, 6, 152, 152, 66, 66, 6, 269, 269, 269, 269, 6, 152, 152, 66, 66, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'pianoroll': array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), 'time_signature': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'h_density_complexity': [0, 1, 0, 0, 1, 0, 0, 0]}


In [8]:
chord_id_duplicates_sequence = val_dataset[6]['harmony_ids']

In [9]:
print(chord_id_duplicates_sequence)

[6, 7, 7, 7, 210, 6, 210, 210, 210, 210, 6, 269, 269, 269, 13, 6, 13, 13, 13, 13, 6, 152, 152, 152, 152, 6, 152, 152, 152, 152, 6, 7, 7, 7, 7, 6, 7, 7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [10]:
chord_ids_sequence = remove_consecutive_duplicates( chord_id_duplicates_sequence )
print(chord_ids_sequence)
chord_ids_sequence = remove_out_of_dict_ids( chord_ids_sequence, chord_id_features )
print(chord_ids_sequence)
unique_chord_ids_sequence = list(set(chord_ids_sequence))
print(unique_chord_ids_sequence)
for c in unique_chord_ids_sequence:
    print(c, tokenizer.ids_to_tokens[c], chord_id_features[c])

[6, 7, 210, 6, 210, 6, 269, 13, 6, 13, 6, 152, 6, 152, 6, 7, 6, 7, 1]
[7, 210, 210, 269, 13, 13, 152, 152, 7, 7]
[7, 13, 269, 210, 152]
7 C:maj [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
13 C:7 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
269 A:min [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
210 G:maj [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1]
152 F:maj [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]


In [11]:
src_id = unique_chord_ids_sequence[0]
dst_id = unique_chord_ids_sequence[1]

src_feat = torch.LongTensor(chord_id_features[src_id])
dst_feat = torch.LongTensor(chord_id_features[dst_id])

feats = compute_edge_features(src_feat, dst_feat)
print(src_id, tokenizer.ids_to_tokens[src_id], src_feat)
print(dst_id, tokenizer.ids_to_tokens[dst_id], dst_feat)
print('Features: ', feats)

7 C:maj tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0])
13 C:7 tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0])
Features:  tensor([0.0000, 0.7500])


In [12]:
# chord_id_duplicates_sequence = [10,11,12,10]
g = make_graph_from_input_ids(chord_id_duplicates_sequence, chord_id_features, use_probabilities=True)

In [13]:
# make graph dataset

graph_val_dataset = []

for d in tqdm(val_dataset):
    chord_id_duplicates_sequence = d['harmony_ids']
    g = make_graph_from_input_ids(
            chord_id_duplicates_sequence,
            chord_id_features,
            use_probabilities=True
        )
    if g is not None:
        graph_val_dataset.append( g )
    else:
        print('Short sequence: ', chord_id_duplicates_sequence)

100%|██████████| 758/758 [00:00<00:00, 6336.03it/s]

Short sequence:  [6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 7, 7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Short sequence:  [6, 4, 4, 4, 4, 6, 4, 4, 4, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [14]:
print(chord_id_duplicates_sequence)
# print(g)
print(g.x)
print(g.node_ids)
print(g.edge_index)
print(g.edge_attr)
print(g.edge_weight)

[6, 269, 269, 269, 269, 6, 123, 123, 123, 123, 6, 123, 123, 123, 123, 6, 269, 269, 269, 269, 6, 269, 7, 7, 7, 6, 66, 66, 66, 66, 6, 123, 123, 123, 123, 6, 269, 269, 269, 269, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
         0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
         0., 1., 0., 0., 0., 0.]])
tensor([ 66, 123, 269,   7])
tensor([[2, 1, 1, 2, 2, 3, 0],
        [1, 1, 2, 2, 3, 0, 1]])
tensor([[0.1667, 0.2000],
        [0.0000, 1.0000],
        [0.1667, 0.2000],
        [0.0000, 1.0000],
        [0.5000, 0.5000],
        [0.3333, 0.000

In [15]:
from models_graph import HarmonicGraphEncoder, BilinearDecoder, HarmonicGAE, edge_index_to_dense

In [16]:
hidden_dim = 64

encoder = HarmonicGraphEncoder(hidden_dim=hidden_dim)
decoder = BilinearDecoder(hidden_dim=hidden_dim)
gae = HarmonicGAE(hidden_dim=hidden_dim)


In [17]:
z = encoder(g)

In [18]:
print(z.shape)

torch.Size([4, 64])


In [19]:
x = decoder(z)

In [20]:
print(x)

tensor([[1.9433e-01, 3.6916e-01, 1.9682e-01, 2.3969e-01],
        [3.6597e-04, 9.8117e-01, 1.6898e-02, 1.5669e-03],
        [2.7751e-03, 9.4293e-01, 4.6293e-02, 8.0066e-03],
        [8.8483e-02, 6.1840e-01, 1.6314e-01, 1.2998e-01]],
       grad_fn=<SoftmaxBackward0>)


In [21]:
y = gae( g )

In [22]:
print(y)

tensor([[0.3673, 0.2027, 0.2055, 0.2244],
        [0.6349, 0.0621, 0.0906, 0.2123],
        [0.4621, 0.1444, 0.1541, 0.2394],
        [0.3283, 0.2435, 0.2146, 0.2136]], grad_fn=<SoftmaxBackward0>)


In [23]:
target = edge_index_to_dense(g)

loss_fn = torch.nn.KLDivLoss(reduction="batchmean")
loss = loss_fn(torch.log(y + 1e-9), target)

In [24]:
print(target)

tensor([[0.0000, 1.0000, 0.0000, 0.0000],
        [0.0000, 0.3333, 0.6667, 0.0000],
        [0.0000, 0.3333, 0.3333, 0.3333],
        [1.0000, 0.0000, 0.0000, 0.0000]])


In [25]:
print(loss.item())

1.311625361442566


In [26]:
g0 = graph_val_dataset[0]

In [27]:
print(g0)

Data(x=[3, 24], edge_index=[2, 3], edge_attr=[3, 2], node_ids=[3], edge_weight=[3])


In [28]:
from torch_geometric.loader import DataLoader

loader = DataLoader(graph_val_dataset, batch_size=32, shuffle=False)

In [29]:
b = next(iter(loader))

In [31]:
print(b)

DataBatch(x=[151, 24], edge_index=[2, 197], edge_attr=[197, 2], node_ids=[151], edge_weight=[197], batch=[151], ptr=[33])


In [ ]:
embeddings = []

for batch in loader:
    emb = encoder(batch)
    embeddings.append(emb.detach())

embeddings = torch.cat(embeddings, dim=0)

In [33]:
print(embeddings.shape)

torch.Size([3751, 64])
